Data Cleaning Workflow:

1. Cleaning Objectives & Scope
2. Column Name Standardization
3. Data Quality Issues Identified
4. Missing Value Treatment
5. Duplicate Handling
6. Data Type Standardization
7. Valid Range Enforcement
8. Outlier Assessment (Not Blind Removal)
9. Categorical Standardization
10. Post-Cleaning Validation
11. Clean Dataset Output

---

# **Cleaning Objective & Scope** 

---

The objective of this notebook is to address identified data quality issues and prepare a clean, consistent, and analysis-ready dataset.

Cleaning actions are limited to structural corrections, normalization, and validation. No feature engineering, aggregation, or analytical transformations are performed at this stage to preserve original business meaning.

In [2]:
#Dependencies
import pandas as pd

---

# **Column Name Standardization** 

---

In [16]:
df = pd.read_csv("../data/raw/consumer_data.csv")
df.columns

Index(['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category',
       'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season',
       'Review Rating', 'Subscription Status', 'Shipping Type',
       'Discount Applied', 'Promo Code Used', 'Previous Purchases',
       'Payment Method', 'Frequency of Purchases'],
      dtype='object')

In [17]:
#using snakecasing format: column_name_example

df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('(', '')  
    .str.replace(')', '')
)

df = df.rename(columns = {'purchase_amount_usd': 'purchase_amount'})

df.head()

,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


Column names were standardized to ensure consistency across SQL, Python, and visualization environments. Formatting normalization improves readability and prevents downstream integration issues.

---

# **Data Quality Issues Identified** 

---

## Data Quality Issues Identified

Based on the data understanding phase, the following quality considerations were identified:

- Minor missing values in review-related attributes
- No potential duplicate transaction records
- Numeric attributes requiring boundary validation
- Inconsistent categorical formatting
- Presence of extreme values requiring assessment

These issues are addressed systematically in subsequent steps.


---

# **Missing Value Treatment** 

---

During Data Understanding [Data Quality Assessment], we found:

1. Review Rating Feature contains null values

Treatment Options:

Numerical Data
    |- Mean : It is affected by outliers
    |- Median : Not affected by outliers [If outlier treatment not done, prefer median]

Cateorical Data
    |- Mode

In [20]:
df['review_rating'] = df.groupby('category')['review_rating'].transform(lambda x: x.fillna(x.median()))
df.isnull().sum()

customer_id               0
age                       0
gender                    0
item_purchased            0
category                  0
purchase_amount           0
location                  0
size                      0
color                     0
season                    0
review_rating             0
subscription_status       0
shipping_type             0
discount_applied          0
promo_code_used           0
previous_purchases        0
payment_method            0
frequency_of_purchases    0
dtype: int64

Missing values were evaluated based on business context and analytical relevance. 

Review rating values exhibited limited missingness and were imputed using the median after grouping it based on category to preserve distribution characteristics.

---

# **Duplicate Handling** 

---

During Data Understanding [Data Quality Assessment], we found:

1. No Duplicate records exist in tha data

In [22]:
df = df.drop_duplicates()
df.duplicated().sum()

np.int64(0)

---

# **Data Type Standardization** 

---

In [23]:
df.head()

,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


In [24]:
df["discount_applied"] = df["discount_applied"].map({"Yes": True, "No": False})
df["promo_code_used"] = df["promo_code_used"].map({"Yes": True, "No": False})
df.head()

,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,True,True,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,True,True,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,True,True,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,True,True,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,True,True,31,PayPal,Annually


In [25]:
df.dtypes

customer_id                 int64
age                         int64
gender                     object
item_purchased             object
category                   object
purchase_amount             int64
location                   object
size                       object
color                      object
season                     object
review_rating             float64
subscription_status        object
shipping_type              object
discount_applied             bool
promo_code_used              bool
previous_purchases          int64
payment_method             object
frequency_of_purchases     object
dtype: object

Attributes representing binary states were standardized into boolean formats to support analytical operations and logical filtering.

---

# **Valid Range Enforcement** 

---

In [26]:
df.shape

(3900, 18)

In [ ]:
df = df[df["age"].between(18, 100)]  #drops rows outside this range
df = df[df["review_rating"].between(1, 5)]
df.shape

(3900, 18)

Numeric attributes were validated against logical and business-defined bounds. Records falling outside acceptable ranges were excluded to ensure analytical reliability.

---

# **Outlier Assessment** 

---

In [31]:
Q1 = df["purchase_amount"].quantile(0.25)
Q3 = df["purchase_amount"].quantile(0.75)
IQR = Q3 - Q1

df["purchase_outlier_flag"] = (
    (df["purchase_amount"] < Q1 - 1.5 * IQR) |
    (df["purchase_amount"] > Q3 + 1.5 * IQR)
)

df


,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases,purchase_outlier_flag
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,True,True,14,Venmo,Fortnightly,False
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,True,True,2,Cash,Fortnightly,False
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,True,True,23,Credit Card,Weekly,False
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,True,True,49,PayPal,Weekly,False
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,True,True,31,PayPal,Annually,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,3896,40,Female,Hoodie,Clothing,28,Virginia,L,Turquoise,Summer,4.2,No,2-Day Shipping,False,False,32,Venmo,Weekly,False
3896,3897,52,Female,Backpack,Accessories,49,Iowa,L,White,Spring,4.5,No,Store Pickup,False,False,41,Bank Transfer,Bi-Weekly,False
3897,3898,46,Female,Belt,Accessories,33,New Jersey,L,Green,Spring,2.9,No,Standard,False,False,24,Venmo,Quarterly,False
3898,3899,44,Female,Shoes,Footwear,77,Minnesota,S,Brown,Summer,3.8,No,Express,False,False,24,Venmo,Weekly,False


Outliers were assessed using interquartile range methodology. Extreme purchase values were flagged for analytical awareness rather than removed, as such values may represent genuine high-value transactions.

---

# **Categorical Standardization** 

---

In [34]:
categorical_cols = df.select_dtypes(include="object").columns

df[categorical_cols] = df[categorical_cols].apply(
    lambda x: x.str.strip().str.title()
)
df.head()

,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases,purchase_outlier_flag
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,True,True,14,Venmo,Fortnightly,False
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,True,True,2,Cash,Fortnightly,False
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,True,True,23,Credit Card,Weekly,False
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,True,True,49,Paypal,Weekly,False
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,True,True,31,Paypal,Annually,False


Categorical attributes were standardized through trimming and consistent casing to prevent artificial category inflation during analysis and visualization.

---

# **Post-Cleaning Validation** 

---

In [35]:
df.isna().sum()

customer_id               0
age                       0
gender                    0
item_purchased            0
category                  0
purchase_amount           0
location                  0
size                      0
color                     0
season                    0
review_rating             0
subscription_status       0
shipping_type             0
discount_applied          0
promo_code_used           0
previous_purchases        0
payment_method            0
frequency_of_purchases    0
purchase_outlier_flag     0
dtype: int64

In [36]:
df.duplicated().sum()

np.int64(0)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             3900 non-null   int64  
 1   age                     3900 non-null   int64  
 2   gender                  3900 non-null   object 
 3   item_purchased          3900 non-null   object 
 4   category                3900 non-null   object 
 5   purchase_amount         3900 non-null   int64  
 6   location                3900 non-null   object 
 7   size                    3900 non-null   object 
 8   color                   3900 non-null   object 
 9   season                  3900 non-null   object 
 10  review_rating           3900 non-null   float64
 11  subscription_status     3900 non-null   object 
 12  shipping_type           3900 non-null   object 
 13  discount_applied        3900 non-null   bool   
 14  promo_code_used         3900 non-null   

In [38]:
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases', 'purchase_outlier_flag'],
      dtype='object')

In [39]:
df.head()

,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases,purchase_outlier_flag
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,True,True,14,Venmo,Fortnightly,False
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,True,True,2,Cash,Fortnightly,False
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,True,True,23,Credit Card,Weekly,False
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,True,True,49,Paypal,Weekly,False
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,True,True,31,Paypal,Annually,False


Post-cleaning validation confirmed resolution of identified quality issues, stable record counts, and consistency of schema across all attributes.


---

# **Clean Dataset Output** 

---

In [41]:
df.to_csv("../data/processed/consumer_data_cleaned.csv", index=False)

#Validation
df1 = pd.read_csv("../data/processed/consumer_data_cleaned.csv")
df1.head()

,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases,purchase_outlier_flag
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,True,True,14,Venmo,Fortnightly,False
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,True,True,2,Cash,Fortnightly,False
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,True,True,23,Credit Card,Weekly,False
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,True,True,49,Paypal,Weekly,False
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,True,True,31,Paypal,Annually,False


The cleaned dataset has been persisted as the standardized analytical source for subsequent exploratory analysis, feature engineering, and reporting workflows.


---

# **Next Steps & Assumptions**

---

Subsequent steps will focus on:

- Initial Exploratory Ananlysis (for insights)
- Feature Engineering
- Post Feature Exploratory Analysis
- SQL Business Queries
- Dashboarding in PowerBI